## Compute the camera calibration matrix and distortion coefficients

In [ ]:
# Source: https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

display_chessboard_corners = True
save_chessboard_corners = False

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        if display_chessboard_corners or save_chessboard_corners:
            cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
        if display_chessboard_corners:
            cv2.imshow('img', img)
            cv2.waitKey(100)
            
        if save_chessboard_corners:
            write_name = 'output_images/corners_found' + str(idx) + '.jpg'
            cv2.imwrite(write_name, img)

cv2.destroyAllWindows()

if objpoints and imgpoints:
    print('Success: Found object points and image points.')
else:
    assert False, 'Failure: Did not find object points and image points.'

In [ ]:
# Source: https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

import pickle
%matplotlib inline

# Test undistortion on an image
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('output_images/test_undistort.jpg', dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump(dist_pickle, open("camera_cal/wide_dist_pickle.p", "wb"))

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

## Apply a distortion correction to raw images.

In [ ]:
# Make a list of calibration images
images = glob.glob('test_images/*.jpg')

f, axes = plt.subplots(len(images), 2, figsize=(16, 6 * len(images)))

undistorted = []

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    axes[idx][0].imshow(img)
    axes[idx][0].set_title('Original Image', fontsize=20)
    axes[idx][1].imshow(dst)
    axes[idx][1].set_title('Undistorted Image', fontsize=20)
    undistorted.append(dst)

## Use color transforms, gradients, etc., to create a thresholded binary image.

In [ ]:
def get_sobel_x(gray, ksize):
    return cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)

def get_sobel_y(gray, ksize):
    return cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)

def get_scaled_sobel_x_y(gray, ksize):
    sobelx = get_sobel_x(gray, ksize)
    sobely = get_sobel_y(gray, ksize)
    import math
    magnitude = np.sqrt((sobelx ** 2) + (sobely ** 2))
    scaled_mag = np.uint8(255 * magnitude / np.max(magnitude))
    return scaled_mag

def abs_sobel_thresh(gray, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    if orient == 'x':
        sobel = get_sobel_x(gray, sobel_kernel)
    elif orient == 'y':
        sobel = get_sobel_y(gray, sobel_kernel)
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

def mag_thresh(gray, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    scaled_mag = get_scaled_sobel_x_y(gray, sobel_kernel)
    # Create a binary mask where mag thresholds are met
    binary = np.zeros_like(scaled_mag)
    binary[(scaled_mag >= mag_thresh[0]) & (scaled_mag <= mag_thresh[1])] = 1
    return binary

# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Take the gradient in x and y separately
    sobelx = get_sobel_x(gray, ksize=sobel_kernel)
    sobely = get_sobel_y(gray, ksize=sobel_kernel)
    # Take the absolute value of the x and y gradients
    abs_sobelx = np.abs(sobelx)
    abs_sobely = np.abs(sobely)
    # Calculate the direction of the gradient 
    direction = np.arctan2(abs_sobely, abs_sobelx)
    # Create a binary mask where direction thresholds are met
    sbinary = np.zeros_like(direction)
    sbinary[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    return sbinary

# Define a function that thresholds the S-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_select(img, h_thresh=(0, 180), l_thresh=(0, 255), s_thresh=(0, 255)):
    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    # Apply thresholds to the channels
    h_binary = np.zeros_like(H)
    h_binary[(H > h_thresh[0]) & (H <= h_thresh[1])] = 1
    l_binary = np.zeros_like(L)
    l_binary[(L > l_thresh[0]) & (L <= l_thresh[1])] = 1
    s_binary = np.zeros_like(S)
    s_binary[(S > s_thresh[0]) & (S <= s_thresh[1])] = 1
    return (h_binary, l_binary, s_binary)

rows_per_img = 3

f, axes = plt.subplots(len(images) * rows_per_img, 4, figsize=(16, 8 * len(images)))

def show_image(axis, img, gray, title):
    if gray:
        axis.imshow(img, cmap='gray')
    else:
        axis.imshow(img)
    axis.set_title(title, fontsize=20)

for idx, img in enumerate(undistorted):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements
    
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(60, 140))
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(60, 140))
    mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(80, 120))
    dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0.9, 1.1))
    
    (h_bin, l_bin, s_bin) = hls_select(img, h_thresh=(18, 25), s_thresh=(80, 255), l_thresh=(200, 255))
    
    h_s = np.zeros_like(h_bin)
    h_s[(h_bin == 1) & (s_bin == 1)] = 1
    l_s = np.zeros_like(l_bin)
    l_s[(l_bin == 1) & (s_bin == 1)] = 1
    mag_dir = np.zeros_like(mag_binary)
    mag_dir[(mag_binary == 1) & (dir_binary == 1)] = 1
    gradxy = np.zeros_like(gradx)
    gradxy[(gradx == 1) & (grady == 1)] = 1
    
    hs_or_ls = np.zeros_like(h_s)
    hs_or_ls[(h_s == 1) | (l_s == 1)] = 1
    all_or = np.zeros_like(h_s)
    all_or[(h_s == 1) | (l_s == 1) | (mag_dir == 1) | (gradxy == 1)] = 1
    
    top = idx * rows_per_img
    mid = idx * rows_per_img + 1
    bottom = idx * rows_per_img + 2
    
    show_image(axes[top][0], img, False, 'Undistorted')
    show_image(axes[top][1], h_bin, True, 'H')
    show_image(axes[top][2], l_bin, True, 'L')
    show_image(axes[top][3], s_bin, True, 'S')
    show_image(axes[mid][0], h_s, True, 'H & S')
    show_image(axes[mid][1], l_s, True, 'L & S')
    show_image(axes[mid][2], mag_dir, True, 'Magnitude & Direction')
    show_image(axes[mid][3], gradxy, True, 'Gradx & Grady')    
    show_image(axes[bottom][0], all_or, True, 'All Or')
    show_image(axes[bottom][1], hs_or_ls, True, '(H & S) | (L & S)')
    show_image(axes[bottom][2], grady, True, 'Grady')
    show_image(axes[bottom][3], grady2, True, 'Grady2')

## TODO: Apply a perspective transform to rectify binary image ("birds-eye view").

## TODO: Detect lane pixels and fit to find the lane boundary.

## TODO: Determine the curvature of the lane and vehicle position with respect to center.

## TODO: Warp the detected lane boundaries back onto the original image.

## TODO: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.